# Week 08 Live Coding Demo — Statistics & Probability II

**Covered today (new examples distinct from slides)**
1. Central Limit Theorem (CLT) — means of exponential waiting times
2. Confidence intervals: Gaussian mean (σ known), Gaussian mean (σ unknown, t‑based), Poisson rate
3. Propagation of uncertainty: kinetic energy K = ½ m v² — linear approx vs Monte Carlo
4. Hypothesis tests: z‑test for sensor bias; Poisson rate increase test with z‑equivalent
5. Likelihoods & likelihood ratio: Gaussian mean and Poisson rate (new datasets); link to z²
6. Correlation & covariance: dual photodiodes and 4‑channel array cross‑talk


## 0) Imports, RNG seed, and plotting defaults

In [ ]:
# Core numerical computing and plotting libraries
import numpy as np, matplotlib.pyplot as plt
# Mathematical functions for statistical calculations
from math import sqrt, pi, exp, factorial
# Statistical distributions for hypothesis testing and confidence intervals
from scipy.stats import norm, t, chi2, poisson

# Set random number generator with fixed seed for reproducible results
rng = np.random.default_rng(808)

# Configure matplotlib for clean, publication-ready plots
plt.rcParams.update({
    "figure.figsize": (6.2, 3.8),  # Standard figure size
    "axes.grid": True,              # Add grid for easier reading
    "axes.spines.top": False,       # Remove top spine for cleaner look
    "axes.spines.right": False,     # Remove right spine for cleaner look
    "font.size": 11,                # Readable font size
})
print("Environment ready — numpy", np.__version__)


## 1) Central Limit Theorem (CLT) — means of exponential waiting times

**Story**
- A detector sees random triggers with waiting time $T\sim\mathrm{Exponential}(\lambda)$ (very non‑Gaussian).
- We average **N** independent waiting times to estimate the mean wait $1/\lambda$.

**Goals**
- Show that the distribution of the **sample mean** becomes approximately Gaussian as N grows.
- Verify the predicted mean and standard deviation $\mu = 1/\lambda,\ \sigma_{\bar T}=\frac{1/\lambda}{\sqrt{N}}$.


In [ ]:
# Parameters for exponential distribution simulation
lam = 0.6  # events per second → true mean wait 1/lam = 1.67 seconds
M   = 20000  # number of repeated "experiments" per N (large for good statistics)

def sample_means(N):
    """Generate M sample means, each from N exponential waiting times.
    
    Args:
        N: Number of exponential samples to average for each mean
        
    Returns:
        Array of M sample means
    """
    # Generate M experiments, each with N exponential waiting times
    # scale=1/lam is the mean of the exponential distribution
    waits = rng.exponential(scale=1/lam, size=(M, N))
    # Average across the N samples for each experiment (axis=1)
    return waits.mean(axis=1)

# Create subplot layout for comparing different sample sizes
fig, ax = plt.subplots(1,3, figsize=(10,3.6))
for j, N in enumerate([2, 5, 20]):
    # Generate sample means for this N
    means = sample_means(N)
    
    # Plot histogram of sample means
    ax[j].hist(means, bins=60, density=True, alpha=0.7, label=f"N={N}")
    
    # Calculate theoretical CLT parameters
    mu  = 1/lam  # True mean of exponential distribution
    sig = (1/lam)/np.sqrt(N)  # Standard error of the mean (CLT prediction)
    
    # Generate points for theoretical Gaussian curve
    xs = np.linspace(mu - 4*sig, mu + 4*sig, 400)
    # Gaussian PDF: (1/σ√(2π)) * exp(-(x-μ)²/(2σ²))
    gauss = (1/(sig*np.sqrt(2*np.pi))) * np.exp(-(xs-mu)**2/(2*sig**2))
    
    # Overlay theoretical Gaussian approximation
    ax[j].plot(xs, gauss, "k--", lw=2, label="Gaussian approx")
    ax[j].set_title(f"mean of N={N} exponential waits")
    ax[j].set_xlabel("sample mean wait [s]"); ax[j].set_ylabel("density")
    ax[j].legend()
plt.tight_layout(); plt.show()


## 2) Confidence intervals (CIs)

We construct intervals that—across many repeated experiments—cover the true parameter at a chosen rate.
We keep **units** and **assumptions** explicit in each example.


### 2a) CI for a Gaussian mean — instrument σ known (clock frequency)

**Story**
- A frequency counter measures a stable reference; manufacturer specifies readout noise $\sigma_{\text{spec}}$.
- We average N readings and quote a 95% CI using the known $\sigma_{\text{spec}}$.

**Assumptions**
- Readout noise is Gaussian with known standard deviation $\sigma_{\text{spec}}$.
- Samples are independent (reasonable for separated readings).


In [ ]:
# True parameters (unknown in real experiments)
mu_true   = 10.000_000  # MHz (true reference frequency, unknown in practice)
sigma_spec = 0.003      # MHz (instrument specification - known from manufacturer)
N = 80                  # number of independent readings

# Simulate measurements: true value + Gaussian noise
x = mu_true + rng.normal(0, sigma_spec, size=N)

# Calculate sample statistics
xbar = x.mean()  # Sample mean
se   = sigma_spec/np.sqrt(N)  # Standard error of the mean (using known σ)

# 95% confidence interval: x̄ ± 1.96 * SE
# 1.96 is the 97.5th percentile of standard normal (two-sided 95% CI)
L, U = xbar - 1.96*se, xbar + 1.96*se

# Visualization
xs = np.linspace(mu_true-0.02, mu_true+0.02, 400)
plt.axvspan(L, U, color="tab:blue", alpha=0.2, label="95% CI")  # CI region
plt.axvline(xbar, color="tab:blue", label=f"mean = {xbar:.6f} MHz")  # Sample mean
plt.axvline(mu_true, color="k", ls="--", label="true (hidden)")  # True value (for comparison)
plt.title("Clock frequency — 95% CI with known σ")
plt.xlabel("frequency [MHz]"); plt.legend(); plt.show()

print(f"95% CI: [{L:.6f}, {U:.6f}] MHz  (half-width {1.96*se:.6f} MHz)")


### 2b) CI for a Gaussian mean — σ unknown (t‑based, small N)

**Story**
- We measure laser line center with a spectrometer (small N). Noise σ is not known a priori.
- Use the **t distribution** with $N-1$ degrees of freedom to inflate uncertainty for small N.

**Assumptions**
- Measurement errors are approximately Gaussian.
- Samples are independent; σ is estimated by the sample standard deviation $s$.


In [ ]:
N = 9  # Small sample size (typical for expensive measurements)
true_wavelength = 632.8  # nm (HeNe laser line, hidden truth)
# Simulate measurements with unknown standard deviation
reads = true_wavelength + rng.normal(0, 0.06, size=N)  # unknown σ≈0.06 nm

# Calculate sample statistics
xbar = reads.mean()  # Sample mean
s = reads.std(ddof=1)  # Sample standard deviation (ddof=1 for unbiased estimate)

# t-distribution critical value for 95% two-sided confidence interval
# df = N-1 degrees of freedom, 0.975 = 97.5th percentile (two-sided 95%)
tcrit = t.ppf(0.975, df=N-1)

# t-based confidence interval: x̄ ± t_crit * s/√N
L, U = xbar - tcrit*s/np.sqrt(N), xbar + tcrit*s/np.sqrt(N)

# Visualization with error bars
plt.errorbar([0], [xbar], yerr=tcrit*s/np.sqrt(N), fmt="o", capsize=6, label="95% CI")
plt.axhline(true_wavelength, color="k", ls="--", label="true (hidden)")
plt.xticks([]); plt.ylabel("wavelength [nm]"); plt.title("t‑interval for mean (σ unknown)")
plt.legend(); plt.show()

print(f"mean={xbar:.3f} nm, s={s:.3f} nm, 95% CI: [{L:.3f}, {U:.3f}] nm")


## 3) Propagation of uncertainty — kinetic energy $K=\tfrac{1}{2}mv^2$

**Story**
- We measure mass $m$ and velocity $v$ (with uncertainties) and report kinetic energy $K$.
- How does measurement uncertainty in $m,v$ propagate to $K$?

**Plan**
1. Linear (first‑order) propagation: $\sigma_K^2 \approx (\tfrac{\partial K}{\partial m})^2\sigma_m^2 + (\tfrac{\partial K}{\partial v})^2\sigma_v^2$ (assuming independence).
2. Monte Carlo: sample $m,v$ from their uncertainties, compute $K$, and compare the histogram to the linear‑Gaussian approximation.


In [ ]:
# True values and measurement uncertainties
m_true, sigma_m = 0.250, 0.002   # kg (mass ± uncertainty)
v_true, sigma_v = 12.0,  0.25    # m/s (velocity ± uncertainty)

# Calculate true kinetic energy K = ½mv²
K = 0.5*m_true*v_true**2

# Linear uncertainty propagation: σ_K² = (∂K/∂m)²σ_m² + (∂K/∂v)²σ_v²
# Partial derivatives of K = ½mv² with respect to m and v
dK_dm = 0.5 * v_true**2  # ∂K/∂m = ½v²
dK_dv = m_true * v_true  # ∂K/∂v = mv
# Linear approximation of uncertainty (assumes independent, Gaussian errors)
sigma_K_lin = np.sqrt((dK_dm*sigma_m)**2 + (dK_dv*sigma_v)**2)

# Monte Carlo simulation: sample from measurement uncertainties
N = 200000  # Large number of samples for good statistics
m = rng.normal(m_true, sigma_m, size=N)  # Sample masses from N(m_true, σ_m)
v = rng.normal(v_true, sigma_v, size=N)  # Sample velocities from N(v_true, σ_v)
K_samples = 0.5*m*v**2  # Calculate K for each (m,v) pair
K_bar, K_std = K_samples.mean(), K_samples.std(ddof=1)  # MC statistics

# Visualization: compare MC distribution with linear Gaussian approximation
plt.hist(K_samples, bins=120, density=True, alpha=0.6, label="MC samples")
# Theoretical Gaussian from linear propagation
xs = np.linspace(K - 5*sigma_K_lin, K + 5*sigma_K_lin, 400)
gauss = (1/(sigma_K_lin*np.sqrt(2*np.pi))) * np.exp(-(xs-K)**2/(2*sigma_K_lin**2))
plt.plot(xs, gauss, "k--", lw=2, label="linear Gaussian approx")
plt.xlabel("kinetic energy K [J]"); plt.ylabel("PDF")
plt.title("Uncertainty propagation to K = ½ m v²")
plt.legend(); plt.show()

print(f"Linear approx: K≈{K:.3f} J, σ_K≈{sigma_K_lin:.3f} J")
print(f"Monte Carlo  : mean≈{K_bar:.3f} J, std≈{K_std:.3f} J")


## 4) Hypothesis testing

We demonstrate two common tests:
- **z‑test (Gaussian, known σ)**: Is a magnetometer biased vs a zero‑field reference?
- **Poisson counting test**: Did the event rate increase above a known background?

We report both p‑values and an equivalent **z‑sigma** (one‑sided) for intuition.


### 4a) z‑test — magnetometer bias check (known σ)

**Story**
- Magnetometer readings near a mu‑metal shield should be zero on average (μ₀=0).
- Manufacturer reports readout $\sigma_{\text{spec}}$; we average N readings and test for bias.

**Test**
- $z = \dfrac{\bar{x} - \mu_0}{\sigma_{\text{spec}}/\sqrt{N}}$, p‑value (two‑sided) = $2\,\Phi(-|z|)$.


In [ ]:
# Test parameters: H0: μ = μ0 (no bias) vs H1: μ ≠ μ0 (bias present)
mu0, sigma_spec, N = 0.0, 0.12, 120  # Null hypothesis, known σ, sample size

# Simulate data with small bias (0.03) plus random noise
x = mu0 + 0.01 + rng.normal(0, sigma_spec, size=N)

# Calculate test statistic
xbar = x.mean()  # Sample mean
# z-statistic: (x̄ - μ₀) / (σ/√N) ~ N(0,1) under H0
z = (xbar - mu0)/(sigma_spec/np.sqrt(N))
# Two-sided p-value: P(|Z| ≥ |z|) = 2 * P(Z ≥ |z|)
p_two = 2*norm.sf(abs(z)) # sf: "survival function" = 1 - CDF(x)

# Visualization of standard normal distribution and test result
xs = np.linspace(-4, 4, 400)
plt.plot(xs, norm.pdf(xs), label="N(0,1)")  # Standard normal PDF
plt.axvline(z, color="r", lw=2, label=f"observed z = {z:.2f}, p={p_two:.3f}")
plt.title("z‑test for bias (two‑sided)")
plt.legend(); plt.show()

print(f"mean={xbar:.3f}, z={z:.2f}, two‑sided p={p_two:.3f}")


### 4b) Poisson counting test — rate increase?

**Story**
- A background monitor historically averages $b=10$ counts/min.
- Today we observe $n=19$ counts in 1 minute. Is this a significant increase?

**Test**
- One‑sided p‑value $p=\mathbb{P}(N\ge n\,|\,\lambda=b)$ under $\mathrm{Poisson}(b)$.
- Convert to a one‑sided z‑sigma: $z = \Phi^{-1}(1-p)$.


In [ ]:
# Test parameters: H0: λ = b (background rate) vs H1: λ > b (rate increase)
b, n = 10.0, 19  # Background rate b=10 counts/min, observed n=19 counts

# One-sided p-value: P(N ≥ n | λ = b) under Poisson(b)
# poisson.sf(n-1, b) = P(N ≥ n) = 1 - P(N ≤ n-1)
p = poisson.sf(n-1, b)
# Convert p-value to equivalent z-score for intuitive interpretation
z_equiv = norm.isf(p)  # Inverse survival function: Φ⁻¹(1-p)

print(f"one‑minute counts: observed n={n}, expected b={b}")
print(f"p = {p:.3f}, one‑sided z ≈ {z_equiv:.2f}σ")

# Visualization of Poisson distribution and test result
k = np.arange(0, 35)  # Range of possible counts
pmf = poisson.pmf(k, b)  # Probability mass function under H0
plt.bar(k, pmf, alpha=0.6, label="H0: Poisson(b=10)")  # Full distribution
plt.bar(k[k>=n], pmf[k>=n], color="crimson", label=f"tail p={p:.3f}")  # Tail region
plt.axvline(n, color="crimson", lw=2, label=f"n={n}")  # Observed value
plt.xlabel("counts in 1 min"); plt.ylabel("PMF")
plt.title("Poisson test for rate increase")
plt.legend(); plt.show()


## 5) Likelihoods and likelihood‑ratio tests (LRT)

We visualize likelihood functions and connect the **z‑test** to an LRT in the Gaussian‑mean case.
New datasets are used to avoid overlap with slides.


### 5a) Gaussian mean (σ known): likelihood curve and LRT equals z²

**Story**
- A Hall probe measures field strength repeatedly; σ is known from calibration.
- We plot the likelihood $L(\mu)$ vs μ, find the MLE, and compute the LRT statistic $S=-2\log\Lambda$.
- In this simple case, $S=z^2$ (numerically equal).

In [ ]:
# Parameters for Gaussian likelihood demonstration
sigma, mu0, N = 0.25, 1.2, 70  # Known σ, null hypothesis μ₀, sample size
# Generate data with small shift from null hypothesis
x = mu0 + 0.07 + rng.normal(0, sigma, size=N)
muhat = x.mean()  # Maximum likelihood estimate

def logL(mu): 
    """Log-likelihood function for Gaussian mean with known σ.
    
    For N independent observations x_i ~ N(μ, σ²) stored in x (nonlocal variable):
    log L(μ) = -½∑(x_i - μ)²/σ² + constant
    """
    return -0.5*np.sum((x-mu)**2)/sigma**2

# Evaluate likelihood over a range of μ values
mus = np.linspace(muhat-1.5*sigma, muhat+1.5*sigma, 400)
logL_vals = np.array([logL(m) for m in mus])
# Normalize to relative likelihood: L(μ)/L(μ̂) = exp(logL(μ) - logL(μ̂))
logL_vals -= logL_vals.max()

# Likelihood ratio test statistic: S = -2 log Λ = -2(logL(μ₀) - logL(μ̂))
S = -2*(logL(mu0) - logL(muhat))
# For Gaussian case, S should equal z² where z is the z-test statistic
z = (muhat - mu0)/(sigma/np.sqrt(N))

# Visualization of likelihood function
plt.plot(mus, np.exp(logL_vals), label="relative likelihood")
plt.axvline(muhat, color="crimson", lw=2, label=f"MLE μ̂={muhat:.3f}")
plt.axvline(mu0, color="k", ls='--', label=f'$\mu_0$={mu0} (hidden)')
plt.xlabel("μ"); plt.ylabel("L(μ)/L(μ̂)"); plt.title("Gaussian likelihood vs μ")
plt.legend(); plt.show()

print(f"LRT S = {S:.3f}, z² = {z**2:.3f}  (should match for this case)")


### 5b) Poisson rate likelihood — skewed shape at low counts

**Story**
- In a short run, only a few decays are observed. Likelihood for the rate λ is asymmetric.

**Goal**
- Plot the relative likelihood $L(\lambda)/L(\hat\lambda)$ and mark the MLE.


In [ ]:
import math
# Poisson counting experiment: n counts observed in time T
n, T = 5, 30.0  # 5 counts in 30 seconds
lam_true = 0.12  # True rate (hidden in real experiments)

# Evaluate likelihood over a range of rate values λ
lam = np.linspace(0.01, 0.6, 400)
# Log-likelihood for Poisson: log L(λ) = n log(λT) - λT - log(n!)
# where n ~ Poisson(λT) and λT is the expected number of counts
logL = n*np.log(lam*T) - lam*T - np.log(math.factorial(n))
# Normalize to relative likelihood: L(λ)/L(λ̂)
logL -= logL.max()

# Maximum likelihood estimate: λ̂ = n/T
lam_hat = n/T

# Visualization of Poisson likelihood function
plt.plot(lam, np.exp(logL), label="relative likelihood")
plt.axvline(lam_hat, color="crimson", lw=2, label=f"MLE λ̂={lam_hat:.3f} s⁻¹")
plt.axvline(lam_true, color="k", ls="--", label=f"true λ (hidden)")
plt.xlabel("λ [s⁻¹]"); plt.ylabel("L(λ)/L(λ̂)")
plt.title("Poisson likelihood for a rate")
plt.legend(); plt.show()


## 6) Correlation and covariance

Two quick physics‑motivated demos:
1. **Dual photodiodes** measuring the same fluctuating light intensity (gain + noise) → correlation and a linear trend.
2. **4‑channel sensor array** with cross‑talk → covariance matrix heatmap.


### 6a) Correlation — dual photodiodes tracking the same source


In [ ]:
N = 400  # Number of simultaneous measurements

# Simulate correlated photodiode measurements
# Common light intensity I (log-normal distribution for realistic intensity variations)
I = rng.lognormal(mean=0.0, sigma=0.3, size=N)
# Photodiode A: linear response to light + independent noise
A = 1.8*I + rng.normal(0, 0.2, size=N)
# Photodiode B: different gain and offset + independent noise
B = 1.5*I + 0.3 + rng.normal(0, 0.25, size=N)

# Calculate Pearson correlation coefficient
r = np.corrcoef(A, B)[0,1]  # Extract correlation from 2x2 correlation matrix

# Linear regression: B = slope*A + intercept
# Set up design matrix [A, 1] for least squares -- Will cover later 
A1 = np.column_stack([A, np.ones_like(A)])
slope, intercept = np.linalg.lstsq(A1, B, rcond=None)[0]

# Visualization of correlation and linear relationship
plt.scatter(A, B, s=16, alpha=0.7, label=f"data (r≈{r:.2f})")
xs = np.linspace(A.min(), A.max(), 100)
plt.plot(xs, slope*xs + intercept, "crimson", lw=2, label=f"fit B≈{slope:.2f}·A+{intercept:.2f}")
plt.xlabel("Photodiode A [a.u.]"); plt.ylabel("Photodiode B [a.u.]")
plt.title("Two sensors viewing the same light source")
plt.legend(); plt.show()


### 6b) Covariance matrix — 4‑channel array with cross‑talk


In [ ]:
N = 1000  # Number of measurements per channel

# Simulate 4-channel sensor array with cross-talk
# Two common signal sources (s1, s2) that affect multiple channels
s1 = rng.normal(size=N)  # Primary signal source
s2 = 0.5*rng.normal(size=N)  # Secondary signal source (weaker)

# Each channel has different sensitivity to common signals + independent noise
# Channel 1: strong response to s1, weak to s2
X1 = 1.0*s1 + 0.3*s2 + rng.normal(scale=0.4, size=N)
# Channel 2: moderate response to s1, negative to s2 (phase difference)
X2 = 0.6*s1 - 0.2*s2 + rng.normal(scale=0.5, size=N)
# Channel 3: weak negative to s1, strong to s2
X3 = -0.3*s1 + 0.9*s2 + rng.normal(scale=0.6, size=N)
# Channel 4: moderate response to both signals
X4 = 0.4*s1 + 0.4*s2 + rng.normal(scale=0.3, size=N)

# Stack channels into 4×N data matrix
X = np.vstack([X1,X2,X3,X4])
# Calculate 4×4 covariance matrix: C[i,j] = Cov(X_i, X_j)
C = np.cov(X)

# Visualization of covariance matrix as heatmap
plt.imshow(C, cmap="coolwarm")  # Red = positive, blue = negative correlation
plt.colorbar(label="covariance")
plt.xticks([0,1,2,3], ["ch1","ch2","ch3","ch4"])
plt.yticks([0,1,2,3], ["ch1","ch2","ch3","ch4"])
plt.title("Covariance matrix — 4‑channel array")
plt.tight_layout(); plt.show()
